In [2]:
#Prepare google drive mount
from google.colab import drive
drive.mount('/content/gdrive')
path = '/content/gdrive/MyDrive'

Mounted at /content/gdrive


Polecenie poniżej wykonuje duże operacje na plikach. Po zakończeniu wypakowywania należy odczekać kilka minut, aby zmiany były widoczne na gdrive

In [3]:
#Unzip dataset and first prepare
!apt install unzip
!unzip /content/gdrive/MyDrive/pokemon-mixer/dataset.zip -d /content/gdrive/MyDrive/pokemon-mixer

Reading package lists... Done
^C
Archive:  /content/gdrive/MyDrive/pokemon-mixer/dataset.zip
replace /content/gdrive/MyDrive/pokemon-mixer/dataset/Abra/0282b2f3a22745f1a436054ea15a0ae5.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
#Install dependencies
!pip install fastai fastbook
!pip install graphviz
!pip install svglib

In [5]:
from fastbook import *
from fastai.vision.widgets import *
from fastai.vision import *
from fastai.metrics import *
#images
import os
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPM
from PIL import Image

In [6]:
#load classes
dataset_path = Path(f'{path}/pokemon-mixer/dataset')
#dynamic import classes
classes = tuple(os.listdir(dataset_path))
print(classes)

('Arbok', 'Beedrill', 'Aerodactyl', 'Abra', 'Chansey', 'Charizard', 'Caterpie', 'Butterfree', 'Arcanine', 'Alakazam', 'Blastoise', 'Bulbasaur', 'Articuno', 'Bellsprout', 'Diglett', 'Dragonair', 'Gyarados', 'Electabuzz', 'Cloyster', 'Gastly', 'Gengar', 'Charmander', 'Golduck', 'Eevee', 'Hitmonlee', 'Dugtrio', 'Gloom', 'Drowzee', 'Dragonite', 'Golem', 'Dratini', 'Clefairy', 'Goldeen', 'Charmeleon', 'Electrode', 'Farfetchd', 'Flareon', 'Graveler', 'Clefable', 'Exeggutor', 'Ditto', 'Fearow', 'Geodude', 'Dewgong', 'Cubone', 'Exeggcute', 'Dodrio', 'Hitmonchan', 'Golbat', 'Haunter', 'Growlithe', 'Grimer', 'Ekans', 'Doduo', 'Lickitung', 'Marowak', 'Mankey', 'Kingler', 'Krabby', 'Koffing', 'Machamp', 'Jynx', 'Jigglypuff', 'Kabutops', 'Hypno', 'Magmar', 'Horsea', 'Jolteon', 'Magnemite', 'Machoke', 'Kadabra', 'Mew', 'Ivysaur', 'Kabuto', 'Machop', 'Metapod', 'Magneton', 'Lapras', 'Magikarp', 'Kakuna', 'Meowth', 'Kangaskhan', 'Onix', 'Parasect', 'MrMime', 'Pidgey', 'Poliwag', 'Poliwrath', 'Pinsir',

Polecenie poniżej wykonuje wiele operacji na plikach (przy pierwszym uruchomieniu). Po wykonaniu komórki należy odczekać kilka minut, aby zmiany były widoczne na gdrive

In [7]:
#prepare data - convert svg, png files to jpg
print('==================================================')
print('Konwertowanie zdjęć....')
print('==================================================')
for dir in classes:
	for file in os.listdir(f'{dataset_path}/{dir}'):
		file_path = (f'{dataset_path}/{dir}/{file}')
		if '.svg' in file:
			jpg_file = file.replace('.svg', '.jpg')
			jpg_path = (f'{dataset_path}/{dir}/{jpg_file}')
			svg_path = (f'{dataset_path}/{dir}/{file}')
			drawing = svg2rlg(svg_path)                         #load svg to memory
			print(f'{dir}/{file} ===> {dir}/{jpg_file}')
			os.remove(svg_path)                                 #delete svg file
			renderPM.drawToFile(drawing, jpg_path, fmt='JPG')   #convert svg to jpg and save
		if '.png' in file:
			jpg_file = file.replace('.png', '.jpg')
			jpg_path = (f'{dataset_path}/{dir}/{jpg_file}')

			png_image = Image.open(file_path).convert('RGBA')               #load png file with alpha
			background = Image.new("RGBA", png_image.size, (255,255,255))   #create white background
			alpha_composite = Image.alpha_composite(background, png_image)  #replace alpha with white color
			alpha_composite_3 = alpha_composite.convert('RGB')              #convert to RGB
			print(f'{dir}/{file} ===> {dir}/{jpg_file}')
			os.remove(file_path)                                            #delete png file
			alpha_composite_3.save(jpg_path, 'JPEG', quality=100)
#load once more classes
classes = tuple(os.listdir(dataset_path))
print('==================================================')
print('KONIEC KONWERTOWANIA ZDJĘĆ...')
print('==================================================')

Konwertowanie zdjęć....
KONIEC KONWERTOWANIA ZDJĘĆ...


In [112]:
data = DataBlock(
  blocks=(ImageBlock, CategoryBlock),
  get_items=get_image_files,
  splitter=RandomSplitter(valid_pct=0.2,seed=42),
  get_y=parent_label,
  item_tfms=Resize(300)
)

In [113]:
data = data.new(
  item_tfms=RandomResizedCrop(224, min_scale=0.5),
  batch_tfms=aug_transforms(mult=0.0, do_flip=False, flip_vert=False, max_rotate=0.0, min_zoom=0.0, max_zoom=0.0, max_lighting=0.0, max_warp=0.0, p_affine=0.0, p_lighting=0.0, xtra_tfms=None, size=None, mode='bilinear', pad_mode='border', align_corners=True, batch=False, min_scale=1.0))


In [114]:
dls = data.dataloaders(dataset_path, bs = 64)

In [115]:
learn = cnn_learner(dls, models.resnet152, metrics = accuracy)

In [116]:
for i in range(15): #20 epok
	print(f'iteracja : {i}')
	learn.fit_one_cycle(1)
	learn.save(f'{path}/pokemon-mixer/models/a3.1.1.{i}')
	learn.export(fname=f'{path}/pokemon-mixer/learned_pickles/model{i}.pkl')

iteracja : 0


epoch,train_loss,valid_loss,accuracy,time
0,2.564890,1.322504,0.692560,02:47


iteracja : 1


epoch,train_loss,valid_loss,accuracy,time
0,1.275296,0.840200,0.786617,02:48


iteracja : 2


...

epoch,train_loss,valid_loss,accuracy,time
0,0.344103,0.627332,0.835751,02:47


iteracja : 13


epoch,train_loss,valid_loss,accuracy,time
0,0.334150,0.624546,0.843238,02:46


iteracja : 14


epoch,train_loss,valid_loss,accuracy,time
0,0.278170,0.630325,0.842770,02:48


In [99]:
learn.fit_one_cycle(1)
learn.save(f'{path}/pokemon-mixer/models/a3.1.1.15')
learn.export(fname=f'{path}/pokemon-mixer/learned_pickles/model15.pkl')

epoch,train_loss,valid_loss,accuracy,time
0,0.231903,0.613820,0.856341,02:38
